In [ ]:
from pynq import Overlay
from pynq import allocate

In [ ]:
import numpy as np
from time import time
import matplotlib.pyplot as plt 

In [ ]:
# validate data
def Validate(ourOutput, golden, size):
    errors = 0
    N, C, D, H, W = size[0], size[1], size[2], size[3], size[4]
    for n in range(N):
        for c in range(C):
            for d in range(D):
                for h in range(H):
                    for w in range(W):
                        pos = n * C*D*H*W + c * D*H*W + d * H*W + h * W + w
                        if ourOutput[pos] != golden[pos]:
                            print(f'[ERROR]  result[{n+1:3.0f}][{c+1:3.0f}][{d+1:3.0f}][{h+1:3.0f}][{w+1:3.0f}] = {ourOutput[pos]:3.0f}, gold: {golden[pos]:3.0f}, error: {100*(int(ourOutput[pos]) - int(golden[pos])) / int(golden[pos]):3.5f}%')
                            errors += 1
    return errors

def Validate_file(ourOutput, golden, size, fp):
    errors = 0
    N, C, D, H, W = size[0], size[1], size[2], size[3], size[4]
    for n in range(N):
        for c in range(C):
            for d in range(D):
                for h in range(H):
                    for w in range(W):
                        pos = n * C*D*H*W + c * D*H*W + d * H*W + h * W + w
                        if ourOutput[pos] != golden[pos]:
                            print(f'[ERROR]  result[{n+1:3.0f}][{c+1:3.0f}][{d+1:3.0f}][{h+1:3.0f}][{w+1:3.0f}] = {ourOutput[pos]:3.0f}, gold: {golden[pos]:3.0f}, error: {100*(int(ourOutput[pos]) - int(golden[pos])) / int(golden[pos]):3.5f}%', file=fp)
                            errors += 1
    return errors


In [ ]:
def CountArr(filename):
    """
    Count the number of data in the file "filename"
    """
    with open(filename) as f:
        num = 0
        line = f.readline()
        while line:
            num += 1
            line = f.readline()
    return num

def LoadArr(filename, type):
    """
    Load the data array from the file "filename" into a PL numpy array of type "type"
    """
    num = CountArr(filename)
    arr = allocate(shape=(num,), dtype=type)
    with open(filename) as f:
        line = f.readline()
        num = 0
        while line:
            arr[num] = type(np.float32(line))
            num += 1
            line = f.readline()
    return arr

def LoadNpArr(filename, type):
    """
    Load the data array from the file "filename" into a PL numpy array of type "type"
    """
    num = CountArr(filename)
    arr = []
    with open(filename) as f:
        line = f.readline()
        num = 0
        while line:
            arr = arr.append(type(np.float32(line)))
            num += 1
            line = f.readline()
    return np.array(arr, dtype=type)

In [ ]:
my_input = LoadArr('layer4.1.conv1.0.0.weight.dat', np.int8)
output = allocate(shape=(5308416,), dtype=np.int8)

In [ ]:
# run the model
# load ip
ol = Overlay("./bitstreams/r2plus1d_io_v3.bit")
ip_r2plus1d = ol.r2plus1d_0

# write input address
ip_r2plus1d.write(0x10, my_input.device_address)
# write output address
ip_r2plus1d.write(0x1C, output.device_address)

In [ ]:
# start computation
# start the computation for hls hardware
timeKernelStart = time()
ip_r2plus1d.write(0x00, 0x01)
# wait for the computation to finish
while (ip_r2plus1d.read(0x00) & 0x4) == 0x0:
    continue
timeKernelEnd = time()
print("hardware execution time: " + str(timeKernelEnd - timeKernelStart) + " s")

In [ ]:
# validate the result
errors = Validate(output, my_input, [1, 5308416, 1, 1, 1])
if errors:
    print("[FAIL] There are some errors QQ, error rate: ", errors / 5308416 * 100, "%")
else:
    print("[PASS] Congratulation! All results are correct")